In [1]:
"""
Contact Avi D Shah at        as2654 at njms.rutgers.edu
Contact Dr. Jason Yang at    jason.y at rutgers.edu

10/1/2022

The purpose of this script is to repair genbank files output by PGAP.

You should first run the final cell to initialize parse_genbank.

"Repairing" genbank files consists of replacing gene fields with 
gene names/synonyms (and locus tags when these are not available),
and numbering repeats (e.g. esxN.k where k is the index of the gene).
Locus tag fields will be replaced with the appropriate Rv# from the 
current Mtb reference made available by the NCBI (NC_000962.3).
"""

import os
import json
import pandas as pd
import scipy
import numpy as np
import multiprocessing
from multiprocessing import Process, Manager
from math import ceil
import time

print(os.getcwd())
print("\n", os.listdir("."))
print("\n", os.listdir("Rv1_09302022"))

/cache/home/as2654/Yang/crossref_annot

 ['esxJ_Alland.faa', 'cross_ref_genes.py', 'repair_pgap_gbk2.ipynb', 'repaired_pgap_03012022.gbff', 'AllandRv_results', '.ipynb_checkpoints', 'Rv1_09302022', 'acc_metadata_save.json', 'Rv1_09302022.zip', 'notebook.tex', 'esxN_Alland.faa', 'repair_pgap_gbk_09302022.ipynb', 'repair_pgap_gbk.ipynb', 'repaired_pgapout.gbff', 'replace_out.txt', 'GCA_000195955.2_ASM19595v2_genomic.fna', 'GCA_000195955.2_ASM19595v2_genomic.gbff', 'AllandRv_results.zip']

 ['cwltool.log', 'annot.gbk', 'annot.fna', 'final_asnval_diag.xml', 'initial_asnval_diag.xml', 'final_asndisc_diag.xml', 'annot.gff', 'annot.faa', 'H37Rv_Dec2019Updated_Three_pilon.fasta', 'initial_asndisc_diag.xml', 'annot.sqn', 'calls.tab']


In [2]:
# methods to parse genbank files

import re

def chunk_genbank(path_to_gbk):
    parsing_mainfile = False
    parsing_tail = False

    header = []
    contents = []
    tail = []

    with open(path_to_gbk, "r") as gbk_read:
        current_record = []
        for line in gbk_read:
            if not parsing_mainfile and not parsing_tail:
                header.append(line)
                if line.startswith("FEATURES"):
                    parsing_mainfile = True
                    continue
            if parsing_mainfile:
                if line.startswith("ORIGIN"):
                    contents.append(current_record)
                    parsing_mainfile = False
                    parsing_tail = True
                else:
                    # start a new record
                    if line.startswith(" " * 5) and not line.startswith(" " * 21):
                        if current_record:
                            contents.append(current_record)
                            current_record = []
                        current_record.append(line)
                    elif line.startswith(" " * 21):
                        current_record.append(line)
            if parsing_tail:
                tail.append(line)
    return header, contents, tail


# utility methods for extracting features of interest
# note, these require the output of parse_genbank for the
# same file to be stored in global var "pgap_out"

import re

def get_range(content_block):
    if type(content_block) is list:
        return re.sub("\s+", " ", content_block[0]).strip().split(" ")[1]
    elif type(content_block) is str: # this contingency can be ignored
        return re.sub("\s+", " ", content_block).strip().split(" ")[1]
    return None

# added support for multiline fields like "inference" and "translation"
def get_field(content_block, field_name, multiline_sep = " "):
    field_found = False
    retstr = ""
    for line_idx in range(len(content_block)):
        line = content_block[line_idx]
        if field_name in line and not field_found:
            field_found = True
            retstr += line.strip().split("=")[1]
        elif field_found:
            if line.strip().startswith("/"):
                return re.sub("\"", "", retstr)
            else:
                retstr += multiline_sep + line.strip()
    return re.sub("\"", "", retstr)


def get_inference_acc(content_block):
    #current_range = get_range(content_block)
    inference_val = get_field(content_block, "inference")
    if "RefSeq" in inference_val:
        return inference_val.split("RefSeq:")[1]
    return None



In [4]:
# get pgap results
outpath_gbk = os.path.join("Rv1_09302022", "annot.gbk")

header   = []
contents = []
tail     = []

header, contents, tail = chunk_genbank(outpath_gbk)

# print out first few entries, see that parser worked
for hl in header[:3]:
    print(hl[:-1])
for content_group in contents[:3]:
    print()
    for cgl in content_group:
        print(cgl[:-1])
for tl in tail[:3]:
    print(tl[:-1])



LOCUS       cluster_001_consensus_pilon_pilon_pil> 4417942 bp    DNA
            circular BCT 30-SEP-2022
DEFINITION  Mycobacterium tuberculosis strain H37Rv chromosome, complete

     source          1..4417942
                     /organism="Mycobacterium tuberculosis"
                     /mol_type="genomic DNA"
                     /strain="H37Rv"
                     /db_xref="taxon:1773"

     gene            1..1524
                     /gene="dnaA"
                     /locus_tag="Rv1_000001"

     CDS             1..1524
                     /gene="dnaA"
                     /locus_tag="Rv1_000001"
                     /inference="COORDINATES: similar to AA
                     sequence:RefSeq:NP_214515.1"
                     /note="Derived by automated computational analysis using
                     gene prediction method: Protein Homology.
                     GO_function: GO:0003688 - DNA replication origin binding
                     [Evidence IEA];
                   

In [6]:
# associate Rv1_00xxxx locus tags with refseq inference accessions

ltags_Rv1 = []

# list all Rv1_00XXXX locus tags in ascending order
# (this *should* just be Rv_00001 to Rv_00xxxx (the highest)
# without any gaps)
for content_group in contents:
    ltag = get_field(content_group, "/locus_tag")
    if ltag and ltag not in ltags_Rv1:
        ltags_Rv1.append(ltag)


# create an association between locus_tags and RefSeq inference accessions
ltag_acc_map = {}
for content_group in contents:
    ltag = get_field(content_group, "/locus_tag")
    acc = get_inference_acc(content_group)
    ltag_acc_map[ltag] = acc

# see the first 30 locus tags and their RefSeq inference accs
for l in ltags_Rv1[:30]:
    print(l)
    print(ltag_acc_map[l])


Rv1_000001
NP_214515.1
Rv1_000002
NP_214516.1
Rv1_000003
NP_214517.1
Rv1_000004
NP_214518.1
Rv1_000005
NP_214519.2
Rv1_000006
NP_214520.1
Rv1_000007
NP_214521.1
Rv1_000008
None
Rv1_000009
None
Rv1_000010
WP_003400294.1
Rv1_000011
None
Rv1_000012
NP_214522.1
Rv1_000013
WP_003400321.1
Rv1_000014
NP_214524.1
Rv1_000015
NP_214525.1
Rv1_000016
WP_003872099.1
Rv1_000017
YP_177615.1
Rv1_000018
NP_214528.1
Rv1_000019
NP_214529.1
Rv1_000020
NP_214530.1
Rv1_000021
NP_214531.1
Rv1_000022
WP_015385224.1
Rv1_000023
NP_214533.1
Rv1_000024
NP_214534.1
Rv1_000025
None
Rv1_000026
NP_214535.1
Rv1_000027
NP_214536.1
Rv1_000028
NP_214537.1
Rv1_000029
NP_214538.1
Rv1_000030
NP_214539.1


In [9]:
from Bio import Entrez

# get info from NCBI; gene names and legacy locus tags for each
# inference RefSeq accession

Entrez.email = "as2654@njms.rutgers.edu"

handle = Entrez.efetch(db="protein", id="NP_214536.1", retmode="xml")
record = Entrez.read(handle)



In [37]:

save = False
print(len(record[0]["GBSeq_feature-table"]))
for val in record[0]["GBSeq_feature-table"]:
    for k, v in val.items():
        if k == "GBFeature_quals":
            for a in v:
                for b, c in a.items():
                    print(b, c)
                print("")
        print("")


4



GBQualifier_name organism
GBQualifier_value Mycobacterium tuberculosis H37Rv

GBQualifier_name strain
GBQualifier_value H37Rv

GBQualifier_name type_material
GBQualifier_value type strain of Mycobacterium tuberculosis

GBQualifier_name db_xref
GBQualifier_value taxon:83332





GBQualifier_name product
GBQualifier_value transcriptional regulator WhiB5

GBQualifier_name calculated_mol_wt
GBQualifier_value 15047





GBQualifier_name region_name
GBQualifier_value Whib

GBQualifier_name note
GBQualifier_value Transcription factor WhiB; pfam02467

GBQualifier_name db_xref
GBQualifier_value CDD:426788





GBQualifier_name gene
GBQualifier_value whiB5

GBQualifier_name locus_tag
GBQualifier_value Rv0022c

GBQualifier_name gene_synonym
GBQualifier_value whmG

GBQualifier_name coded_by
GBQualifier_value complement(NC_000962.3:27023..27442)

GBQualifier_name experiment
GBQualifier_value DESCRIPTION:Gene expression, mutation analysis[PMID:22733573]

GBQualifier_name experiment
GBQualifier_